In [3]:
import numpy as np

ahp_matrix = np.array([
    [1,   1, 1/2,   4,   2],
    [1,   1, 1/2,   4,   2],
    [2,   2,   1,   8,   4],
    [1/4, 1/4, 1/8,   1, 1/2],
    [1/2, 1/2, 1/4,   2,   1]
])

n = ahp_matrix.shape[0]
R = np.zeros((n, n, 2))

def ahp_to_ifr(ahp_value):
    if ahp_value >= 1:
        mu = np.log(ahp_value + 1) / np.log(9 + 1)  # log scaling
        mu = min(mu, 0.9)
    else:
        mu = ahp_value * 0.5
    
    nu = 1 - mu - 0.1  # Ensure hesitancy
    nu = max(nu, 0)
    return [mu, nu]

for i in range(n):
    for j in range(n):
        if i == j:
            R[i, j] = [1.0, 0.0]  # Diagonal: full membership
        else:
            R[i, j] = ahp_to_ifr(ahp_matrix[i, j])

# Ensure reciprocity: R[j,i] = [ν, μ] if R[i,j] = [μ, ν]
for i in range(n):
    for j in range(i+1, n):
        mu, nu = R[i, j]
        R[j, i] = [nu, mu]

stakeholders = ["S1", "S2", "S3", "S4", "S5"]
print("Intuitionistic Fuzzy Relation Matrix R:")
for i in range(n):
    print(f"\n{stakeholders[i]}:")
    for j in range(n):
        mu, nu = R[i, j]
        print(f"  {stakeholders[j]}: [{mu:.2f}, {nu:.2f}]", end="")
    print()

print("\nR = np.array([")
for i in range(n):
    row = "    ["
    for j in range(n):
        row += f"[{R[i,j,0]:.2f}, {R[i,j,1]:.2f}]"
        if j < n-1:
            row += ", "
    row += "]"
    if i < n-1:
        row += ","
    print(row)
print("])")

Intuitionistic Fuzzy Relation Matrix R:

S1:
  S1: [1.00, 0.00]  S2: [0.30, 0.60]  S3: [0.25, 0.65]  S4: [0.70, 0.20]  S5: [0.48, 0.42]

S2:
  S1: [0.60, 0.30]  S2: [1.00, 0.00]  S3: [0.25, 0.65]  S4: [0.70, 0.20]  S5: [0.48, 0.42]

S3:
  S1: [0.65, 0.25]  S2: [0.65, 0.25]  S3: [1.00, 0.00]  S4: [0.90, 0.00]  S5: [0.70, 0.20]

S4:
  S1: [0.20, 0.70]  S2: [0.20, 0.70]  S3: [0.00, 0.90]  S4: [1.00, 0.00]  S5: [0.25, 0.65]

S5:
  S1: [0.42, 0.48]  S2: [0.42, 0.48]  S3: [0.20, 0.70]  S4: [0.65, 0.25]  S5: [1.00, 0.00]

R = np.array([
    [[1.00, 0.00], [0.30, 0.60], [0.25, 0.65], [0.70, 0.20], [0.48, 0.42]],
    [[0.60, 0.30], [1.00, 0.00], [0.25, 0.65], [0.70, 0.20], [0.48, 0.42]],
    [[0.65, 0.25], [0.65, 0.25], [1.00, 0.00], [0.90, 0.00], [0.70, 0.20]],
    [[0.20, 0.70], [0.20, 0.70], [0.00, 0.90], [1.00, 0.00], [0.25, 0.65]],
    [[0.42, 0.48], [0.42, 0.48], [0.20, 0.70], [0.65, 0.25], [1.00, 0.00]]
])


In [5]:
def construct_perfect_multiplicative_consistent_ifpr(R):
    """
    Constructs the Perfect Multiplicative Consistent IFPR (R_bar) from the original IFPR R.
    
    For i < k, if we define the chain over t = i+1, ..., k-1, then:
    
       μ̃(i,k) = { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) } /
                  { [∏ₜ ( μ(i,t) * μ(t,k) )]^(exponent) + [∏ₜ ( (1-μ(i,t)) * (1-μ(t,k)) )]^(exponent) }
    
       ν̃(i,k) is defined similarly using ν.
    
    The exponent is chosen as follows:
      - If chain length = (k - i - 1) equals 1, exponent = 1.
      - If chain length > 1, exponent = 0.5 (i.e. take square root).
    
    For adjacent indices (k = i+1), the original values are kept.
    The lower-triangular part is filled by reciprocity:
      R_bar[k,i] = (ν̃(i,k), μ̃(i,k))
    """
    n = R.shape[0]
    R_bar = np.zeros((n, n, 2), dtype=float)

    # 1. Set diagonal entries to (0.5, 0.5)
    for i in range(n):
        R_bar[i, i] = [0.5, 0.5]

    # 2. Compute upper-triangular entries
    for i in range(n):
        for k in range(i+1, n):
            if k == i + 1:
                # For adjacent indices, simply use the original value.
                R_bar[i, k] = R[i, k]
            else:
                exponent = 1.0 / (k - i - 1)
                # Compute chain-products for membership
                prod_mu = 1.0
                prod_1mu = 1.0
                for t in range(i+1, k):
                    prod_mu   *= (R[i, t, 0] * R[t, k, 0])
                    prod_1mu  *= ((1 - R[i, t, 0]) * (1 - R[t, k, 0]))
                left_mu  = prod_mu ** exponent
                right_mu = prod_1mu ** exponent
                denom_mu = left_mu + right_mu
                new_mu = left_mu / denom_mu if denom_mu != 0 else 0.5

                # Compute chain-products for non-membership
                prod_nu = 1.0
                prod_1nu = 1.0
                for t in range(i+1, k):
                    prod_nu   *= (R[i, t, 1] * R[t, k, 1])
                    prod_1nu  *= ((1 - R[i, t, 1]) * (1 - R[t, k, 1]))
                left_nu  = prod_nu ** exponent
                right_nu = prod_1nu ** exponent
                denom_nu = left_nu + right_nu
                new_nu = left_nu / denom_nu if denom_nu != 0 else 0.5

                R_bar[i, k] = [new_mu, new_nu]

    # 3. Fill the lower-triangular part using reciprocity:
    # If R_bar[i,k] = (μ, ν), then set R_bar[k,i] = (ν, μ).
    for i in range(n):
        for k in range(i+1, n):
            mu_val, nu_val = R_bar[i, k]
            R_bar[k, i] = [nu_val, mu_val]

    return R_bar

# Construct the Perfect Multiplicative Consistent IFPR
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)

# Print results with better formatting
np.set_printoptions(precision=4, suppress=True)
print("Original R (upper triangle):")
print(R[..., 0], "\n", R[..., 1], "\n")  # Print membership (μ) and non-membership (ν) matrices

print("Perfect Multiplicative Consistent IFPR (R_bar):")

# Dynamically determine the size of the matrix
n = R.shape[0]  # Number of rows (and columns, since it's an N x N matrix)

# Loop through each row and column dynamically
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_bar[i, k, 0]
        nu_val = R_bar[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print(f"Row {i+1}:", "\t".join(row_str))

Original R (upper triangle):
[[1.     0.301  0.25   0.699  0.4771]
 [0.599  1.     0.25   0.699  0.4771]
 [0.65   0.65   1.     0.9    0.699 ]
 [0.201  0.201  0.     1.     0.25  ]
 [0.4229 0.4229 0.201  0.65   1.    ]] 
 [[0.     0.599  0.65   0.201  0.4229]
 [0.301  0.     0.65   0.201  0.4229]
 [0.25   0.25   0.     0.     0.201 ]
 [0.699  0.699  0.9    0.     0.65  ]
 [0.4771 0.4771 0.699  0.25   0.    ]] 

Perfect Multiplicative Consistent IFPR (R_bar):
Row 1: (0.5000, 0.5000)	(0.3010, 0.5990)	(0.1255, 0.7350)	(0.6340, 0.0000)	(0.3817, 0.3829)
Row 2: (0.5990, 0.3010)	(0.5000, 0.5000)	(0.2500, 0.6500)	(0.7500, 0.0000)	(0.4363, 0.3185)
Row 3: (0.7350, 0.1255)	(0.6500, 0.2500)	(0.5000, 0.5000)	(0.9000, 0.0000)	(0.7500, 0.0000)
Row 4: (0.0000, 0.6340)	(0.0000, 0.7500)	(0.0000, 0.9000)	(0.5000, 0.5000)	(0.2500, 0.6500)
Row 5: (0.3829, 0.3817)	(0.3185, 0.4363)	(0.0000, 0.7500)	(0.6500, 0.2500)	(0.5000, 0.5000)


In [7]:
def check_consistency(R, R_bar, tau=0.1):
    """
    Computes the distance d between the original IFPR matrix R and 
    the perfectly multiplicative consistent IFPR matrix R_bar using:
    
      d(R, R_bar) = (1 / ((n-1) * (n-2))) * sum_{i=1}^{n} sum_{k=i+1}^{n} 
                     [|mu_bar(i,k) - mu(i,k)| + |nu_bar(i,k) - nu(i,k)| + |pi_bar(i,k) - pi(i,k)|],
                     
    where for each entry,
      pi(i,k) = 1 - mu(i,k) - nu(i,k) and
      pi_bar(i,k) = 1 - mu_bar(i,k) - nu_bar(i,k).
    
    Parameters:
      R      : Original IFPR matrix (n x n x 2 numpy array)
      R_bar  : Consistent IFPR matrix (n x n x 2 numpy array)
      tau    : Threshold for acceptable consistency (default: 0.1)
    
    Returns:
      (bool) True if d < tau, else False.
    """
    n = R.shape[0]
    total_diff = 0.0
    
    # Loop over the upper triangular indices (i < k)
    for i in range(n):
        for k in range(i+1, n):
            mu = R[i, k, 0]
            nu = R[i, k, 1]
            pi = 1 - mu - nu
            
            mu_bar = R_bar[i, k, 0]
            nu_bar = R_bar[i, k, 1]
            pi_bar = 1 - mu_bar - nu_bar
            
            diff = abs(mu_bar - mu) + abs(nu_bar - nu) + abs(pi_bar - pi)
            total_diff += diff
    
    # Use normalization factor as in the paper: (n-1)*(n-2) or 2*(n-1)(n-2) idk man
    denominator = (n - 1) * (n - 2)
    d = total_diff / denominator
    
    print("Distance d(R, R_bar) =", d)
    return d < tau
R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    
# Check consistency:
consistent = check_consistency(R, R_bar, tau=0.1)
if consistent:
    print("The IFPR is consistent (d < 0.1).")
else:
    print("The IFPR is NOT consistent (d >= 0.1).")

Distance d(R, R_bar) = 0.17885243662599595
The IFPR is NOT consistent (d >= 0.1).


In [11]:
def repair_ifahp_algorithm_2(R, sigma=0.5, tau=0.1, max_iter=100):
    """
    Repairs the IFPR R using the fused intuitionistic preference algorithm (Algorithm 2)
    until the consistency measure d(R, R_bar) < tau.
    
    The update rules for off-diagonal entries are:
    
      μ^(p+1)(i,k) = [ (μ^(p)(i,k))^(1-σ) * (μ̄(i,k))^σ ] /
                      { (μ^(p)(i,k))^(1-σ) * (μ̄(i,k))^σ + (1-μ^(p)(i,k))^(1-σ) * (1-μ̄(i,k))^σ }
    
      ν^(p+1)(i,k) = [ (ν^(p)(i,k))^(1-σ) * (ν̄(i,k))^σ ] /
                      { (ν^(p)(i,k))^(1-σ) * (ν̄(i,k))^σ + (1-ν^(p)(i,k))^(1-σ) * (1-ν̄(i,k))^σ }
    
    Diagonal entries are fixed to (0.5, 0.5) and the lower-triangular part is
    updated via reciprocity.
    """
    # First, compute the perfect IFPR matrix.
    R_bar = construct_perfect_multiplicative_consistent_ifpr(R)
    print("Initial consistency check:")
    if check_consistency(R, R_bar, tau):
        print("R is already consistent with R_bar; no repair needed.")
        return R

    # Start the iterative repair process.
    R_current = R.copy()
    n = R_current.shape[0]
    
    for p in range(max_iter):
        R_next = np.copy(R_current)
        
        # Update all off-diagonal entries using the ratio-based formulas.
        for i in range(n):
            for k in range(n):
                if i != k:
                    mu_p = R_current[i, k, 0]
                    nu_p = R_current[i, k, 1]
                    mu_bar = R_bar[i, k, 0]
                    nu_bar = R_bar[i, k, 1]
                    
                    # Membership update:
                    num_mu = (mu_p ** (1 - sigma)) * (mu_bar ** sigma)
                    den_mu = num_mu + ((1 - mu_p) ** (1 - sigma)) * ((1 - mu_bar) ** sigma)
                    mu_next = num_mu / den_mu if den_mu != 0 else 0.5
                    
                    # Non-membership update:
                    num_nu = (nu_p ** (1 - sigma)) * (nu_bar ** sigma)
                    den_nu = num_nu + ((1 - nu_p) ** (1 - sigma)) * ((1 - nu_bar) ** sigma)
                    nu_next = num_nu / den_nu if den_nu != 0 else 0.5
                    
                    R_next[i, k, 0] = mu_next
                    R_next[i, k, 1] = nu_next
        
        # Enforce the diagonal to remain (0.5, 0.5)
        for i in range(n):
            R_next[i, i] = [0.5, 0.5]
        
        # Enforce reciprocity for the lower-triangular part:
        for i in range(n):
            for k in range(i+1, n):
                mu_val, nu_val = R_next[i, k]
                R_next[k, i] = [nu_val, mu_val]
        
        print(f"Iteration {p+1} consistency check:")
        if check_consistency(R_next, R_bar, tau):
            print(f"Repair successful after {p+1} iterations.")
            return R_next
        
        R_current = R_next
    
    print("Max iterations reached; returning final repaired matrix.")
    return R_current
R_repaired = repair_ifahp_algorithm_2(R, sigma=0.8, tau=0.1, max_iter=100)
    
# --- Print the repaired IFPR matrix ---
print("\nRepaired IFPR matrix:")
for i in range(n):
    row_str = []
    for k in range(n):
        mu_val = R_repaired[i, k, 0]
        nu_val = R_repaired[i, k, 1]
        row_str.append(f"({mu_val:.4f}, {nu_val:.4f})")
    print("Row", i+1, ":", "\t".join(row_str))
    

Initial consistency check:
Distance d(R, R_bar) = 0.17885243662599595
Iteration 1 consistency check:
Distance d(R, R_bar) = 0.017846566909744628
Repair successful after 1 iterations.

Repaired IFPR matrix:
Row 1 : (0.5000, 0.5000)	(0.3010, 0.5990)	(0.1452, 0.7191)	(0.6475, 0.0000)	(0.4003, 0.3908)
Row 2 : (0.5990, 0.3010)	(0.5000, 0.5000)	(0.2500, 0.6500)	(0.7403, 0.0000)	(0.4444, 0.3383)
Row 3 : (0.7191, 0.1452)	(0.6500, 0.2500)	(0.5000, 0.5000)	(0.9000, 0.0000)	(0.7403, 0.0000)
Row 4 : (0.0000, 0.6475)	(0.0000, 0.7403)	(0.0000, 0.9000)	(0.5000, 0.5000)	(0.2500, 0.6500)
Row 5 : (0.3908, 0.4003)	(0.3383, 0.4444)	(0.0000, 0.7403)	(0.6500, 0.2500)	(0.5000, 0.5000)


In [13]:
def compute_criterion_weights(R):
    """
    Computes the intuitionistic fuzzy weights ω_i = (μ_i, ν_i) for each row i
    using Formula (26).
    """
    n = R.shape[0]
    weights = np.zeros((n, 2), dtype=float)
    
    # Calculate the denominators first (they are the same for all weights)
    sum_all_mu = 0.0
    sum_all_one_minus_nu = 0.0
    
    for i in range(n):
        for k in range(n):
            sum_all_mu += R[i, k, 0]  # Sum of all μ(i,k)
            sum_all_one_minus_nu += (1.0 - R[i, k, 1])  # Sum of all (1-ν(i,k))
    
    # Now calculate weights for each criterion
    for i in range(n):
        sum_mu_i = sum(R[i, k, 0] for k in range(n))  # Sum of μ(i,k) for row i
        sum_one_minus_nu_i = sum(1.0 - R[i, k, 1] for k in range(n))  # Sum of (1-ν(i,k)) for row i
        
        # Calculate weights according to formula 26
        if sum_all_one_minus_nu <= 1e-15 or sum_all_mu <= 1e-15:
            w_mu = 0.5
            w_nu = 0.5
        else:
            w_mu = sum_mu_i / sum_all_one_minus_nu
            w_nu = 1.0 - (sum_one_minus_nu_i / sum_all_mu)
        
        weights[i, 0] = w_mu
        weights[i, 1] = w_nu
    
    return weights
weights = compute_criterion_weights(R_repaired)

print("\nCriterion Weights (μ, ν):")
for i, (mu_w, nu_w) in enumerate(weights, start=1):
    print(f"ω_{i} = ({mu_w:.4f}, {nu_w:.4f})")


Criterion Weights (μ, ν):
ω_1 = (0.1391, 0.7383)
ω_2 = (0.1768, 0.6990)
ω_3 = (0.2448, 0.6152)
ω_4 = (0.0523, 0.8535)
ω_5 = (0.1311, 0.7501)


In [15]:
def convert_to_crisp_weights(weights):
    """
    Converts intuitionistic fuzzy weights to crisp weights.
    
    Args:
    - weights: np.ndarray of shape (n, 2) where weights[i, 0] is μ_i and weights[i, 1] is ν_i
    
    Returns:
    - crisp_weights: Normalized crisp weights
    """
    n = weights.shape[0]
    crisp_values = np.zeros(n)
    
    for i in range(n):
        mu = weights[i, 0]
        nu = weights[i, 1]
        
        # Score function: S(ω_i) = μ_i - ν_i
        score = mu - nu
        
        # Accuracy function: H(ω_i) = μ_i + ν_i
        accuracy = mu + nu
        
        # Combined score (this is one approach)
        crisp_values[i] = 0.5 * (score + 1) * (1 + accuracy - score)
    
    # Normalize to ensure sum is 1
    sum_crisp = np.sum(crisp_values)
    if sum_crisp > 0:
        crisp_weights = crisp_values / sum_crisp
    else:
        # Equal weights if all values are 0
        crisp_weights = np.ones(n) / n
        
    return crisp_weights
crisp_weights = convert_to_crisp_weights(weights)
print(crisp_weights)

[0.1972 0.2276 0.279  0.1069 0.1892]
